In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
from matplotlib import rc
from parcels import plotTrajectoriesFile
from parcels.plotting import create_parcelsfig_axis, cartopy_colorbar
import cartopy
%pylab inline

In [ ]:
class ParticleData(object):
    def __init__(self):
        self.id = []

def load_particles_file(fname, varnames):
    T = ParticleData()
    pfile = xr.open_dataset(str(fname), decode_cf=True)
    T.id = pfile.variables['trajectory'][:]

    for v in varnames:
        setattr(T, v,  np.ma.filled(pfile.variables[v][:], np.nan))
    pfile.close()
    
    return T

T = load_particles_file("medforams.nc", ['lon', 'lat', 'time', 'temp', 'salt'])
I, J =np.where(T.temp < -99)
T.temp[I, J] = np.nan
I, J =np.where(T.salt < -99)
T.salt[I, J] = np.nan

In [ ]:
from matplotlib import animation, rc
pylab.rcParams['figure.figsize'] = (11, 8)
projection = cartopy.crs.PlateCarree()

for varname in ['salt', 'temp']:
    var = getattr(T, varname)
    fig, ax = plt.subplots(1, 1,subplot_kw={'projection': projection})
    ax.coastlines(resolution='50m')
    ax.add_feature(cartopy.feature.LAND)

    plottimes = np.arange(np.min(T.time), np.max(T.time), dtype='datetime64[1D]')

    b = np.where(np.logical_and(T.time <= plottimes[-1], T.time > plottimes[-2]))
    scat = ax.scatter(T.lon[b], T.lat[b], s=3, c=var[b],
                      transform=cartopy.crs.Geodetic())
    scat.set_clim(np.nanmin(var), np.nanmax(var))
    cb = plt.colorbar(scat, orientation="horizontal")
    axextent = [-6, 38, 30, 46]
    ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())

    ttl = ax.set_title(varname+' of particles on ' + str(plottimes[0]))
    frames = np.arange(len(plottimes)-1, 0, -1)

    def animate(t):
        b = np.where(np.logical_and(T.time <= plottimes[t], T.time > plottimes[t-1]))
        scat.set_offsets(np.vstack((T.lon[b], T.lat[b])).transpose())
        scat.set_array(var[b])
        ttl.set_text(varname+' of particles on ' + str(plottimes[t]))
        return scat,

    ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=True)
    anim
    anim.save("medforams_%s.gif" %varname, writer='imagemagick', fps=10)